# Predicting Demonstration Size on Demographic Characteristics and Salience

## GW DATS 6103: Introduction to Data Mining Final Project
### Name: Alexander D. Silberman

Note: Data is limited to 2017–2019, as COVID may substantially throw off the data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from census import Census
from us import states

import censusdis.data as ced
from censusdis.datasets import ACS1_PROFILE
from censusdis import states

## Data Acquisition and Wrangling

### Results Dataset: Crowd Counting Consortium Phase 1 (2017-2020)

Our results dataset: Crowd Counting Consortium Phase 1 (2017-2020): https://ash.harvard.edu/programs/crowd-counting-consortium/

In [86]:
crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")
crowd_data.head(5)

C:\Users\alexa\AppData\Local\Temp\ipykernel_15548\206043875.py:1: DtypeWarning: Columns (6,22,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  crowd_data = pd.read_csv("C:/Users/alexa/Code/GW DATS/6103 12 Intro to Data Mining/Final Project/data/crowd_counting_consortium_2017-2020/ccc_compiled_20172020.csv",encoding_errors="replace")


,date,locality,state,location_detail,online,type,macroevent,actors,claims,valence,...,source_28,source_29,source_30,notes,lat,lon,resolved_locality,resolved_county,resolved_state,fips_code
0,2017-01-01,Washington,DC,Lafayette Square Park,0.0,vigil,NaN,NaN,"for banning nuclear weapons, for peace",0.0,...,NaN,NaN,NaN,White House Peace Vigil continuous since June ...,38.907192,-77.036871,Washington,District of Columbia,DC,11001.0
1,2017-01-01,Mankato,MN,NaN,0.0,vigil,NaN,Peace Vigil Mankato,for peace,0.0,...,NaN,NaN,NaN,every Sunday since 2001,44.163578,-93.999400,Mankato,Blue Earth County,MN,27013.0
2,2017-01-01,Minneapolis,MN,U.S. Bank Stadium,0.0,protest; banner drop,NaN,general protestors,"against the Dakota Access Pipeline, for indige...",1.0,...,NaN,NaN,NaN,hung banner from stadium roof during NFL game,44.977753,-93.265011,Minneapolis,Hennepin County,MN,27053.0
3,2017-01-01,Little Compton,RI,Town Green,0.0,vigil,NaN,Sakonnet Peace Alliance,"for peace, for gun control, for climate action",1.0,...,NaN,NaN,NaN,every Sunday since 2003,41.510103,-71.171156,Little Compton,Newport County,RI,44005.0
4,2017-01-01,Oak Ridge,TN,Y-12 National Security Complex,0.0,vigil,NaN,Oak Ridge Environmental Peace Alliance,for abolishing nuclear weapons,0.0,...,NaN,NaN,NaN,every Sunday since the late 1990s,36.010356,-84.269645,Oak Ridge,Anderson County,TN,47001.0


In [87]:
# source_cols = np.array(["source_"]*30) + np.arange(1,31).astype(str)
# crowd_data.drop(columns=source_cols, inplace=True)
# crowd_data.head(5)


### American Community Survey

In [173]:
api_key = "29e7dfea2f8b253a0a10ccd9626f78e49f4f0a4f"

# census_vars = {
#     "NAME":"name",
#     "DP05_0001E":"total_population",
# }

census_vars = ced.variables.search(ACS1_PROFILE, 2017)
census_vars = census_vars[census_vars["GROUP"] != "N/A"]
census_vars

census_data = [ced.download(ACS1_PROFILE, year, pd.concat([pd.Series(["NAME"]), census_vars["VARIABLE"]], ignore_index=True), 
                           metropolitan_statistical_area_micropolitan_statistical_area ="*", 
                           api_key=api_key,
                           ).set_index("METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STATISTICAL_AREA")# .rename(columns=census_vars)
               for year in range(2017,2020)]


census_data = pd.concat(census_data, keys = np.arange(2017,2020))
census_data

NAME  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                                                 
2017 44220                                                                 Springfield, OH Metro Area   
     44300                                                               State College, PA Metro Area   
     44340                                                                  Statesboro, GA Micro Area   
     44420                                                         Staunton-Waynesboro, VA Metro Area   
     44620                                                               Stevens Point, WI Micro Area   
...                                                                                               ...   
2019 49620                                                                York-Hanover, PA Metro Area   
     49660                                               Youngstown-Warren-Boardman, OH-PA Metro Area   
     49700                                                                   Yuba City, CA Metro Area   
     49740                                                                        Yuma, AZ Metro Area   
     49780                                                                  Zanesville, OH Micro Area   

                                                         DP02PR_0001E  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                 
2017 44220                                                        NaN   
     44300                                                        NaN   
     44340                                                        NaN   
     44420                                                        NaN   
     44620                                                        NaN   
...                                                               ...   
2019 49620                                                        NaN   
     49660                                                        NaN   
     49700                                                        NaN   
     49740                                                        NaN   
     49780                                                        NaN   

                                                         DP02PR_0001PE  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                  
2017 44220                                                         NaN   
     44300                                                         NaN   
     44340                                                         NaN   
     44420                                                         NaN   
     44620                                                         NaN   
...                                                                ...   
2019 49620                                                         NaN   
     49660                                                         NaN   
     49700                                                         NaN   
     49740                                                         NaN   
     49780                                                         NaN   

                                                         DP02PR_0002E  \
     METROPOLITAN_STATISTICAL_AREA_MICROPOLITAN_STAT...                 
2017 44220                                                        NaN   
     44300                                                        NaN   
     44340                                                        NaN   
     44420                                                        NaN   
     44620                                                        NaN   
...                                                               ...   
2019 49620                                                        NaN   
     49660                                                        NaN   
     49700                                                        NaN   
     49740     

### Most Important Problem Dataset, Second Release (2024)

In [ ]:
issue_data, meta_issue_data = pd.read_stata("C:\\Users\\alexa\\Code\\GW DATS\\6103 12 Intro to Data Mining\\Final Project\data\\2024-MIPD\\USMISC2024-MIPD_ind.dta", chunksize=10000)